# MVE Axiom Prediction

% https://colab.research.google.com/drive/1DIQm9rOx2mT1bZETEeVUThxcrP1RKqAn#scrollTo=IS1dPinuyPCy reference

In [1]:
#%env CUDA_VISIBLE_DEVICES=""

# TODO




* Try to get a reasonable score?
* TODO add batchnorm [done]
* TODO add residual connections [done]
* TODO add other directions? [done]
* TODO add FileWriter [done]
* TODO separate itno functions [done]

# Imports

In [1]:
import torch
from torch.nn import Embedding
import torch.nn.functional as F

from torch_geometric.nn import GCNConv, Linear
import torch.nn as nn
import torch_geometric.nn as pyg_nn
from torch_geometric.transforms import to_undirected, ToUndirected

In [2]:
import sys, os
from pathlib import Path

sys.path.append(str(Path(os.path.abspath("")).parent))

import config
from dataset import get_data_loader, BenchmarkType
from stats_writer import Writer
from dataset import LearningTask

In [3]:
from dataset import TorchMemoryDataset

In [5]:
TorchMemoryDataset(VAL_ID,  BenchmarkType("deepmath"), **kwargs)

NameError: name 'kwargs' is not defined

In [6]:
from torch_geometric.transforms import ToUndirected


In [44]:
d = TorchMemoryDataset(VAL_ID,  BenchmarkType("deepmath"), **{"transform": ToUndirected()})

In [45]:
d[1]

Data(x=[203], edge_index=[2, 692], premise_index=[2], conjecture_index=[1], name='t30_glib_001', y=[2])

In [28]:
d = TorchMemoryDataset(VAL_ID,  BenchmarkType("deepmath"), transform= None)

In [46]:
d

TorchMemoryDataset(2465)

In [47]:
d = TorchMemoryDataset(VAL_ID,  BenchmarkType("deepmath"), **{})

In [48]:
d[0].is_directed()

True

In [49]:
d[0].is_undirected()

False

## CONSTANTS

In [4]:
# TRAIN_ID = '../id_files/train.txt'
TRAIN_ID = "../id_files/validation.txt"

VAL_ID = "../id_files/validation.txt"
BENCHMARK_TYPE = BenchmarkType("deepmath")

In [5]:
torch.manual_seed(1234567)

## Load dataset

In [6]:
# dataset_params = {'transform': ToUndirected()}
dataset_params = {"transform": None}

# transform = None

In [7]:
data_instance = get_data_loader(TRAIN_ID, BENCHMARK_TYPE, **dataset_params)
# val_data = get_data_loader(VAL_ID, BENCHMARK_TYPE, **dataset_params)

Dataset: TorchMemoryDataset(2465)


## Data point check

In [8]:
next(iter(data_instance))

DataBatch(x=[36912], edge_index=[2, 60406], premise_index=[842], conjecture_index=[64], name=[64], y=[842], batch=[36912], ptr=[65])

In [9]:
data = next(iter(data_instance))[0]
print(data)
print(data.keys)
print(data.num_nodes)
print(data.num_edges)
print(data.num_node_features)
print(data.has_isolated_nodes())
print(data.is_directed())

Data(x=[164], edge_index=[2, 258], premise_index=[4], conjecture_index=[1], name='t104_zfmisc_1', y=[4])
['y', 'edge_index', 'x', 'conjecture_index', 'premise_index', 'name']
164
258
1
False
True


## Model

In [10]:
def get_dense_output_network(no_dense_layers, hidden_dim, task, dropout_rate):
    # TODO set TYPE
    if task == "premise":
        return DenseOutput(hidden_dim, no_dense_layers, hidden_dim, dropout_rate)
    elif task == "similarity":
        return DenseOutput(hidden_dim * 2, no_dense_layers, hidden_dim, dropout_rate)
    else:
        raise ValueError(f"No dense output network for: {task}")

In [11]:
class DenseOutput(torch.nn.Module):
    def __init__(self, input_dim: int, no_dense_layers: int, hidden_dim: int, dropout_rate: float):
        super(DenseOutput, self).__init__()

        self.input_dim = input_dim
        self.no_dense_layers = no_dense_layers
        self.hidden_dim = hidden_dim
        self.dropout_rate = dropout_rate

        self._build_network()

    def _build_network(self):

        if self.no_dense_layers <= 0:  # No hidden layer
            self.out = nn.Linear(self.input_dim, 1)
        else:
            # Add input layer
            self.lin = nn.ModuleList()
            self.lin.append(nn.Linear(self.input_dim, self.hidden_dim))
            # Ad dense layers
            for _ in range(self.no_dense_layers - 1):
                self.lin.append(nn.Linear(self.hidden_dim, self.hidden_dim))
            # Add output layer
            self.out = nn.Linear(self.hidden_dim, 1)

    def forward(self, x):

        # TODO move this to Stack Model?
        # Extract premises
        # x = x[premise_index]

        # Dense feedforward
        for i in range(self.no_dense_layers):
            x = self.lin[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout_rate, training=self.training)

        # Output layer
        x = self.out(x)
        x = x.squeeze(-1)

        return x

In [12]:
GCN_NORMALISATION = {"batch": nn.BatchNorm1d, "layer": nn.LayerNorm}

In [13]:
def build_normalisation_layers(normaliser, hidden_dim, num_normalisation_layers):

    lns = nn.ModuleList()
    for _ in range(num_normalisation_layers):
        lns.append(normaliser(hidden_dim))
    return lns


def build_conv_model(num_convolutional_layers, hidden_dim, flow):

    convs = nn.ModuleList()
    for _ in range(num_convolutional_layers):
        convs.append(get_conv_layer(hidden_dim, hidden_dim, flow))

    return convs


def get_conv_layer(input_dim, hidden_dim, flow):
    return pyg_nn.GCNConv(input_dim, hidden_dim, flow=flow)


def build_merge_linear_layers(num_linear_layers, hidden_dim):

    lin = nn.ModuleList()
    for _ in range(num_linear_layers):
        lin.append(Linear(hidden_dim * 2, hidden_dim))

    return lin

In [14]:
class GCNDirectional(torch.nn.Module):
    def __init__(self, hidden_dim, num_convolutional_layers, dropout_rate, normalisation, skip_connection):
        super(GCNDirectional, self).__init__()

        self.flow = "target_to_source"  # Sets direction to bottom up

        # Set variables
        self.hidden_dim = hidden_dim
        self.num_convolutional_layers = num_convolutional_layers
        self.dropout_rate = dropout_rate
        self.skip_connection = skip_connection

        # Add convolutional layers
        self.convs = build_conv_model(num_convolutional_layers, hidden_dim, self.flow)

        # Add normalisation layers used in between graph convolutions
        if normalisation is None:
            self.lns = None
        else:
            self.normaliser = GCN_NORMALISATION[normalisation]
            self.lns = build_normalisation_layers(self.normaliser, hidden_dim, num_convolutional_layers - 1)

    def forward(self, x, edge_index):

        # Iterate over each convolutional sequence
        for i in range(self.num_convolutional_layers):

            conv_out = self.convs[i](x, edge_index)
            # Check if applying skip connection
            if self.skip_connection:
                x = x + conv_out
            else:
                x = conv_out

            emb = x
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout_rate, training=self.training)

            if self.lns is not None and not i == self.num_convolutional_layers - 1:  # Apply normalisation
                x = self.lns[i](x)

        return emb, x


sub = GCNDirectional(
    hidden_dim=32, num_convolutional_layers=3, dropout_rate=0.25, normalisation="batch", skip_connection=False
)
sub

GCNDirectional(
  (convs): ModuleList(
    (0): GCNConv(32, 32)
    (1): GCNConv(32, 32)
    (2): GCNConv(32, 32)
  )
  (lns): ModuleList(
    (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [15]:
class GCNBiDirectional(torch.nn.Module):
    def __init__(self, hidden_dim, num_convolutional_layers, dropout_rate, normalisation, skip_connection):
        super(GCNBiDirectional, self).__init__()

        # Set variables
        self.hidden_dim = hidden_dim
        self.num_convolutional_layers = num_convolutional_layers
        self.dropout_rate = dropout_rate
        self.skip_connection = skip_connection

        # Add convolutional layers
        self.convs_up = build_conv_model(num_convolutional_layers, hidden_dim, flow="source_to_target")
        self.convs_down = build_conv_model(num_convolutional_layers, hidden_dim, flow="target_to_source")

        # Add normalisation layers used in between graph convolutions
        if normalisation is None:
            self.lns = None
        else:
            self.normaliser = GCN_NORMALISATION[normalisation]
            self.lns = build_normalisation_layers(self.normaliser, hidden_dim, num_convolutional_layers - 1)

        # Add Linear layers
        self.linear = build_merge_linear_layers(num_convolutional_layers, hidden_dim)

    def forward(self, x, edge_index):

        # Iterate over each convolutional sequence
        for i in range(self.num_convolutional_layers):

            # Apply convolutions
            x_up = self.convs_up[i](x, edge_index)
            x_down = self.convs_down[i](x, edge_index)

            # Check if applying skip connection
            if self.skip_connection:
                x_up = x + x_up
                x_down = x + x_down

            # Concat convolutions
            x = torch.cat((x_up, x_down), dim=1)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout_rate, training=self.training)

            # Merge through linear
            x = self.linear[i](x)
            emb = x
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout_rate, training=self.training)

            # Normalise, if set
            if self.lns is not None and not i == self.num_convolutional_layers - 1:  # Apply normalisation
                x = self.lns[i](x)

        return emb, x


sub = GCNBiDirectional(
    hidden_dim=32, num_convolutional_layers=3, dropout_rate=0.25, normalisation="batch", skip_connection=False
)
sub

GCNBiDirectional(
  (convs_up): ModuleList(
    (0): GCNConv(32, 32)
    (1): GCNConv(32, 32)
    (2): GCNConv(32, 32)
  )
  (convs_down): ModuleList(
    (0): GCNConv(32, 32)
    (1): GCNConv(32, 32)
    (2): GCNConv(32, 32)
  )
  (lns): ModuleList(
    (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (linear): ModuleList(
    (0): Linear(64, 32, bias=True)
    (1): Linear(64, 32, bias=True)
    (2): Linear(64, 32, bias=True)
  )
)

In [16]:
# Add a superclass as the wrapper? Only really need to change the foward function right? And add some more stuff to the output?

In [17]:
class GNNStack(torch.nn.Module):
    def __init__(
        self,
        hidden_dim,
        num_convolutional_layers,
        no_dense_layers,
        direction,
        dropout_rate=0.0,
        task="premise",
        normalisation="layer",
        skip_connection=True,
    ):
        super(GNNStack, self).__init__()

        # Set variables
        self.task = task
        self.dropout_rate = dropout_rate
        self.hidden_dim = hidden_dim

        # Add embedding layer
        self.node_embedding = Embedding(len(config.NODE_TYPE), hidden_dim)

        # Add GCN layer
        if direction == "single":
            gcn_base = GCNDirectional
        elif direction == "separate":
            gcn_base = GCNBiDirectional
        else:
            raise ValueError(f"Unkown gcn direction {direction}")

        self.gcn = gcn_base(
            hidden_dim=self.hidden_dim,
            num_convolutional_layers=num_convolutional_layers,
            dropout_rate=self.dropout_rate,
            normalisation=normalisation,
            skip_connection=skip_connection,
        )

        # Post-message-passing
        self.post_mp = get_dense_output_network(
            no_dense_layers, hidden_dim, task=self.task, dropout_rate=self.dropout_rate
        )

    def forward(self, data):
        x, edge_index, premise_index = data.x, data.edge_index, data.premise_index

        x = self.node_embedding(x)

        emb, x = self.gcn(x, edge_index)

        if self.task == "premise":
            x = self.post_mp(x, premise_index)

            # elif self.task == 'graph':
            #    x = pyg_nn.global_mean_pool(x, batch)
        else:
            raise NotImplementedError()

        return emb, x

    # def loss(self, pred, label):
    #    return F.nll_loss(pred, label)


test_model = GNNStack(
    hidden_dim=32,
    num_convolutional_layers=3,
    no_dense_layers=1,
    direction="single",
    dropout_rate=0.25,
    task="premise",
)

In [18]:
print(test_model)

GNNStack(
  (node_embedding): Embedding(15, 32)
  (gcn): GCNDirectional(
    (convs): ModuleList(
      (0): GCNConv(32, 32)
      (1): GCNConv(32, 32)
      (2): GCNConv(32, 32)
    )
    (lns): ModuleList(
      (0): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    )
  )
  (post_mp): DenseOutput(
    (lin): ModuleList(
      (0): Linear(in_features=32, out_features=32, bias=True)
    )
    (out): Linear(in_features=32, out_features=1, bias=True)
  )
)


In [19]:
# TODO add Graph difference AND add extra network

In [21]:
torch.nn.functional.mse_loss

<function torch.nn.functional.mse_loss(input: torch.Tensor, target: torch.Tensor, size_average: Optional[bool] = None, reduce: Optional[bool] = None, reduction: str = 'mean') -> torch.Tensor>

In [22]:
model = GNNStackSiamese(
    hidden_dim=32,
    num_convolutional_layers=3,
    no_dense_layers=1,
    direction="single",
    dropout_rate=0.25,
    task="premise",
)

{'hidden_dim': 32, 'num_convolutional_layers': 3, 'no_dense_layers': 1, 'direction': 'single', 'dropout_rate': 0.25, 'task': 'premise'}


In [23]:
model.task

'similarity'

Lets only have one output layer for now?

## Train and Test

In [24]:
def train(model, train_data, criterion, optimizer):

    model.train()

    for data in train_data:  # Iterate in batches over the training dataset.
        _, out = model(data)  # Perform a single forward pass. TODO change this

        # loss = criterion(out, data.y)  # Compute the loss.
        loss = criterion(out, data.y, reduction="mean")  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.


def test(model, test_data, criterion, writer, testing: bool):

    model.eval()

    correct = 0
    total_samples = 0
    total_loss = 0

    for data in test_data:  # Iterate in batches over the training/test dataset.
        _, out = model(data)

        pred = torch.sigmoid(out).round().long()
        # correct += data.y.eq(pred).sum().item()

        loss = criterion(out, data.y, reduction="sum")
        correct = loss

        total_loss += loss
        total_samples += len(pred)

    acc_score = correct / total_samples  # Derive ratio of correct predictions.
    total_loss /= total_samples
    total_loss = total_loss.item()

    # print(total_loss)
    acc_score = total_loss

    if testing:
        writer.report_val_score(acc_score)
        writer.report_val_loss(total_loss)
    else:
        writer.report_train_score(acc_score)
        writer.report_train_loss(total_loss)

    return acc_score

In [25]:
train_data = get_data_loader(TRAIN_ID, BENCHMARK_TYPE, task=LearningTask.SIMILARITY, **dataset_params)
val_data = get_data_loader(VAL_ID, BENCHMARK_TYPE, task=LearningTask.SIMILARITY, **dataset_params)

Unsupervised dataset: 100
Unsupervised dataset: 100


In [26]:
LearningTask.SIMILARITY

<LearningTask.SIMILARITY: 'similarity'>

In [27]:
F.mse_loss

<function torch.nn.functional.mse_loss(input: torch.Tensor, target: torch.Tensor, size_average: Optional[bool] = None, reduce: Optional[bool] = None, reduction: str = 'mean') -> torch.Tensor>

In [30]:
# def train_loop():

"""
model = GNNStack(
        hidden_dim=16,
        num_convolutional_layers=1,
        no_dense_layers=1,
        direction="single",
        dropout_rate=0.2,
        task="premise",
)
"""
writer = Writer(model)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
# criterion = torch.nn.BCEWithLogitsLoss(reduction="mean")
# criterion = torch.nn.BCEWithLogitsLoss(reduction="mean")

# criterion = torch.nn.MSELoss(reduction="mean")

criterion = F.mse_loss

# criterion = torch.nn.BCEWithLogitsLoss(reduction="none")


print("\nTraining")
for epoch in range(1, 20):

    train(model, train_data, criterion, optimizer)

    train_score = test(model, train_data, criterion, writer, testing=False)
    test_score = test(model, val_data, criterion, writer, testing=True)
    print(f"Epoch: {epoch:03d}, Train Score: {train_score:.4f}, Test Score: {test_score:.4f}")

    writer.on_step()


# return writer
# writer = train_loop()


Training
Epoch: 001, Train Score: 1613.3157, Test Score: 1613.3157
Epoch: 002, Train Score: 977.2343, Test Score: 977.2342
Epoch: 003, Train Score: 527.2117, Test Score: 527.2117
Epoch: 004, Train Score: 812.5084, Test Score: 812.5084
Epoch: 005, Train Score: 1410.8558, Test Score: 1410.8560
Epoch: 006, Train Score: 1041.7841, Test Score: 1041.7841
Epoch: 007, Train Score: 604.8157, Test Score: 604.8156
Epoch: 008, Train Score: 504.1052, Test Score: 504.1052
Epoch: 009, Train Score: 521.9355, Test Score: 521.9355
Epoch: 010, Train Score: 517.0912, Test Score: 517.0912
Epoch: 011, Train Score: 503.1153, Test Score: 503.1153
Epoch: 012, Train Score: 527.1135, Test Score: 527.1135
Epoch: 013, Train Score: 601.9360, Test Score: 601.9360
Epoch: 014, Train Score: 672.5765, Test Score: 672.5765
Epoch: 015, Train Score: 661.6836, Test Score: 661.6836
Epoch: 016, Train Score: 605.3612, Test Score: 605.3611
Epoch: 017, Train Score: 551.2723, Test Score: 551.2723
Epoch: 018, Train Score: 519.051

In [ ]:
d.edge_index

In [ ]:
d.x_t_batch

In [ ]:
val_data = get_data_loader(VAL_ID, BENCHMARK_TYPE, task=LearningTask.PREMISE, **dataset_params)

In [ ]:
type(next(iter(val_data)).x)

In [ ]:
torch.Tensor

In [ ]:
"""
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.node_embedding = Embedding(len(config.NODE_TYPE), hidden_channels)


        self.conv1 = GCNConv(hidden_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        
        self.linear = Linear(hidden_channels, 1)
        

    def forward(self, input_batch):
        
        x = input_batch.x
        edge_index = input_batch.edge_index
        premise_index = input_batch.premise_index    

        x = self.node_embedding(x)

        x = self.conv1(x, edge_index)
        x = x.relu()
        
        x = F.dropout(x, p=0.5, training=self.training) # TODO add dropout parameter?
        
        x = self.conv2(x, edge_index)
        x = x.relu()
        
        x = x[premise_index]
        x = self.linear(x)
        
        # Remove inner axis
        x = x.squeeze(-1)
        
    
        return x

    
model = GCN(hidden_channels=16)
print(model)
"""